# Evaluate file

In [1]:
from class_utils import *
import pickle


In [28]:
iteration =  "not-critical"
filepath = "/Users/amyburkhardt/Documents/NLP/Code/combining-machine-qual/"
ev_filename = "{}/new_data/i".format(filepath)

# maps estimator model to pickled scikit-learn model file name:
eval_dict = {
    "SVC-not-critical": "{}/saved_models/best_svc_{}.pickle".format(filepath,iteration)
}

In [29]:
# load estimator models from disk
models = {}
for model, file in eval_dict.items():
    with open(file, 'rb') as f:
        loaded_model = pickle.load(f)
    models[model] = loaded_model

In [30]:
models

{'SVC-not-critical': Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.45555555555555555, max_features=None,
         min_df=1, ngram_range=(1, 3),
         preprocessor=<function no...,
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))])}

In [31]:
# get results for each estimatator model
results = {}
for model, estimator in models.items():
    results[model] = evaluate_file(ev_filename, models[model])

In [32]:
# print result counts

for model, result in results.items():
    nneg = len(result[0])
    npos = len(result[1])
    print("Counts for %s:" % model)
    print("Number of NEG docs: %d" % nneg)
    print("Number of POS docs: %d" % npos)
    print("Percent POS: %.2f" % (npos/(npos+nneg)*100))
    print()

Counts for SVC-not-critical:
Number of NEG docs: 11874
Number of POS docs: 1227
Percent POS: 9.37



In [33]:
# first three results of each class
for model, result in results.items():
    print("Top 3 of each class for %s:" % model)
    for c in result.keys():
        print("Class %s: " % str(c))
        for tweet in result[c][:3]:
            print(tweet[1])
        print()

Top 3 of each class for SVC-not-critical:
Class 0: 
Jeb Bush should 'rethink' running for President. Jeb Bush: Parents should 'rethink' opting out of Common Core tests http://www.washingtonexaminer.com/jeb-bush-parents-should-rethink-opting-out-of-common-core-tests/article/2563854#.VUMUB9QfOTo.twitter …

Legal Implications of High-Stakes Assessment: What States [ #SmarterBalanced & #PARCC] Should Know // v ERIC @scoopit http://sco.lt/8YyD5N

Fairtest Reacts to Ongoing Computer-Administered #Testing Problems: Exams Crash in 9 States http://sco.lt/9My8cD  // #SBAC #education


Class 1: 
One wonders why #ohleg Sen. GOP thought it a good idea to release #parcc recs on eve of #NFLDraft. But here they are http://bit.ly/1GAnDS0

We want to be able to offer a good strong education for the students" -Reviving Rapid City Schools on the #optout"

Opt Numbers for Math test Continue to grow on Long Island | Diane Ravitch's blog http://dianeravitch.net/2015/04/30/opt-numbers-for-math-test-continue-t

In [36]:
# save results to disk
for model, result in results.items():
    for c in result.keys():
        out_filename = "%sclassified_new_data/%s_%s_out_class%d.txt" % (filepath,iteration, model, c)
        with open(out_filename, 'w') as f:
            for line in result[c]:
                f.write("%s\t%s" % line)